In [1]:
import datetime as dt
from collections import OrderedDict
from pathlib import Path
import sys

ROOT = Path('../')
DATA_DIR = ROOT/'tests'/'data'
OUT_DIR = ROOT/'output'

sys.path.append(str(ROOT))

%load_ext autoreload
%autoreload 2

import countdowner as cd

In [2]:
# Read a watchlist

w = cd.read_watchlist(DATA_DIR/'watchlist.yaml')
w


{'email_addresses': ['brainbummer@mailinator.com', 'rhymedude@mailinator.com'],
 'name': 'test',
 'products':       description       stock_code
 0  organic cheese           281739
 1    GB chocolate           260803
 2            fake  fake_stock_code}

In [3]:
# Get and parse a product

code = w['products']['stock_code'].iat[-1]
response = cd.get_product(code)
cd.parse_product(response)

OrderedDict([('stock_code', 'fake_stock_code'),
             ('name', None),
             ('description', None),
             ('size', None),
             ('sale_price', None),
             ('price', None),
             ('discount_percentage', None),
             ('unit_price', None),
             ('datetime', '2017-10-06T13:08:24')])

In [4]:
# Get and parse many products

codes = w['products']['stock_code'].values
%time f = cd.collect_products(codes, async=False)
f

CPU times: user 196 ms, sys: 0 ns, total: 196 ms
Wall time: 1.28 s


stock_code                                               name  \
0           281739         Mainland Cheese Block Organic Cheddar 500g   
1           260803  Green & Blacks Chocolate Block Organic Dark Ch...   
2  fake_stock_code                                               None   

                                         description  size  sale_price  price  \
0  Mainland organic cheddar is a mild cheddar che...  500g         NaN  11.00   
1                                               None  100g        3.29   3.89   
2                                               None  None         NaN    NaN   

   discount_percentage  unit_price            datetime  
0                  NaN  $22.00/1KG 2017-10-06 13:08:27  
1            15.424165  $3.29/100G 2017-10-06 13:08:27  
2                  NaN        None 2017-10-06 13:08:28

In [5]:
# Get many products asynchronously and parse

codes = w['products']['stock_code'].values
%time f = cd.collect_products(codes, async=True)
f

CPU times: user 188 ms, sys: 4 ms, total: 192 ms
Wall time: 681 ms


stock_code                                               name  \
0           260803  Green & Blacks Chocolate Block Organic Dark Ch...   
1           281739         Mainland Cheese Block Organic Cheddar 500g   
2  fake_stock_code                                               None   

                                         description  size  sale_price  price  \
0                                               None  100g        3.29   3.89   
1  Mainland organic cheddar is a mild cheddar che...  500g         NaN  11.00   
2                                               None  None         NaN    NaN   

   discount_percentage  unit_price            datetime  
0            15.424165  $3.29/100G 2017-10-06 13:08:31  
1                  NaN  $22.00/1KG 2017-10-06 13:08:31  
2                  NaN        None 2017-10-06 13:08:31

In [6]:
# Filter sales

g = cd.filter_sales(f)
g

name  sale_price  price  \
0  Green & Blacks Chocolate Block Organic Dark Ch...        3.29   3.89   

   discount_percentage  
0            15.424165

In [9]:
# Email sales

domain = cd.get_secret('mailgun_domain')  # replace with your Mailgun domain
key = cd.get_secret('mailgun_api_key')  # replace with your Mailgun API key
cd.email(g, w['email_addresses'], domain, key)

<Response [200]>

In [7]:
# Get products in bulk

cd.run_pipeline(DATA_DIR/'watchlist.yaml')

stock_code                                               name  \
0           260803  Green & Blacks Chocolate Block Organic Dark Ch...   
1           281739         Mainland Cheese Block Organic Cheddar 500g   
2  fake_stock_code                                               None   

                                         description  size  sale_price  price  \
0                                               None  100g        3.29   3.89   
1  Mainland organic cheddar is a mild cheddar che...  500g         NaN  11.00   
2                                               None  None         NaN    NaN   

   discount_percentage  unit_price            datetime  
0            15.424165  $3.29/100G 2017-10-06 13:09:33  
1                  NaN  $22.00/1KG 2017-10-06 13:09:33  
2                  NaN        None 2017-10-06 13:09:33

In [14]:
# Run pipeline again: read watchlist, collect products, write products to file, 
# filter sales, and email results

cd.run_pipeline(DATA_DIR/'watchlist.yaml', OUT_DIR/'products.csv', domain, key)
%less {OUT_DIR/'products.csv'}